In [1]:
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')

from ase.units import Bohr
import numpy as np
import qml_interface as qmi
import geometry_euston as ge
import glob
import scipy as sc
from matplotlib import pyplot as plt



import alchemy_tools2 as at
import utils_qm as uq

def get_alchpot_free_fit(molecule, poly_obs):
    """
    calculates the alchemical binding potential for every atom in molecule
    """
    # construct h matrix
    h_mat = ge.abc_to_hmatrix(20,20,20,90,90,90)/Bohr
    # get alchpot of free atoms
    alch_pot_free = []
    for atom_I in molecule:
        alch_pot_free_I = 0
        for atom_J in molecule:
            dist = ge.distance_pbc(atom_I[1:4], atom_J[1:4], h_mat)
                    
            nuc_charge = atom_J[0]
            poly_obj = poly_obs[nuc_charge] # dictionary
            elec_pot = poly_obj.__call__([dist])[0]
            alch_pot_free_I += elec_pot
        alch_pot_free.append(alch_pot_free_I)
    alch_pot_free = np.array(alch_pot_free)
    return(alch_pot_free)

def get_alch_bind_pot(molecule, poly_obs):
    """
    calculates the alchemical binding potential for every atom in molecule
    """
    alch_pot_free = get_alchpot_free_fit(molecule, poly_obs)
    alch_pot_bind = molecule[:, 4] - alch_pot_free
    return(alch_pot_bind)

def get_element_charge(el):
    if el == 'H':
        return(1)
    elif el == 'C':
        return(6)
    elif el == 'N':
        return(7)
    elif el == 'O':
        return(8)
    else:
        raise ValueError('Symbol for given charge not available')

def load_poly_objs(paths_poly):
    poly_objs = dict()
    for p in paths_poly:
        el = p.split('/')[-1].split('_')[-1]
        charge = get_element_charge(el)
        poly_objs[charge] = uq.load_obj(p)
    return(poly_objs)

def get_alchpot_free(nuclei, densities_free_atoms, meshgrid, h_matrix, pos_free_atom = np.array([10.0, 10, 10])/Bohr):
    """
    calculate alchemical potential of free atoms at position of every nucleus
    length units should be given in Bohr
    """
    alch_pot_free = []
    for atom_I in nuclei:
        alch_pot_free_I = 0
        for atom_J in nuclei:
            # get density of free atom J
            nuc_charge = atom_J[0]
            density_fa = densities_free_atoms[nuc_charge]
            
            # calculate distance of R_I to all gridpoints (shift because free J is in center of box)
            s = (pos_free_atom - atom_J[1:4])
            RI_prime = atom_I[1:4] + s
            dist = at.distance_MIC2(RI_prime, meshgrid, h_matrix)
            # integrate
            elec_pot = -(density_fa/dist).sum()

            alch_pot_free_I += elec_pot
            
        alch_pot_free.append(alch_pot_free_I)
        
    return(np.array(alch_pot_free))

def get_meshgrid(lx, ly, lz, nx, ny, nz):
    """
    returns the coordinates of the grid points where the density values are given as a meshgrid
    works so far only for orthogonal coordinate axes
    """
    # length along the axes
    l_x = lx[0]*nx
    l_y = ly[1]*ny
    l_z = lz[2]*nz
    # gpts along every axis
    x_coords = np.linspace(0, l_x-lx[0], nx)
    y_coords = np.linspace(0, l_y-ly[1], ny)
    z_coords = np.linspace(0, l_z-lz[2], nz)
    # create gridpoints
    meshgrid = np.meshgrid(x_coords, y_coords, z_coords, indexing='ij')
    return(meshgrid)

In [19]:
# explicit

# load average density of free atoms
base_path = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/'
densities_free_atoms = {1.0:None, 6.0:None, 7.0:None, 8.0:None}
elements = ['H', 'C', 'N', 'O']
for el, k in zip(elements, densities_free_atoms):
    densities_free_atoms[k] = uq.load_obj(base_path + f'av_dens_{el}')

# get meshgrid and h_matrix
#  cell_parameters
nx, ny, nz = densities_free_atoms[1.0].shape
lx, ly, lz = np.array([[20/(Bohr*nx), 0 , 0], [0, 20/(Bohr*ny), 0], [0, 0, 20/(Bohr*nz)]])

h_matrix = [lx*nx, ly*ny, lz*nz]
meshgrid = get_meshgrid(lx, ly, lz, nx, ny, nz)

compound_path = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_000274/'
molecule = np.loadtxt(compound_path + 'atomic_energies_with_mic.txt')


alchpot_free_explicit= get_alchpot_free(molecule, densities_free_atoms, meshgrid, h_matrix)

In [20]:
alchpot_free_explicit

array([-6.30634156, -7.0606324 , -7.39112216, -7.41569327, -6.56818392,
       -6.70726212, -5.24422945, -5.58060647, -5.46210059, -6.156898  ,
       -6.15757575, -6.30547768, -6.49797604, -6.4215648 , -5.81078825,
       -5.57179625, -5.63291808, -5.90932731, -5.66250377, -5.92040317])

In [2]:
# from fit
compound_path = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_000274/'
molecule = np.loadtxt(compound_path + 'atomic_energies_with_mic.txt')

# debug get_alchpot_free
paths_poly = ['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/H/poly_H',
              '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/C/poly_C',
              '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/N/poly_N',
              '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/free_atoms/O/poly_O']

poly_objs = dict()
for k,p in zip([1.0, 6.0, 7.0, 8.0], paths_poly):
    poly_objs[k] = uq.load_obj(p)
molecule_binding = np.loadtxt(compound_path + 'atomic_binding_energies.txt')
alchpot_free_fit = get_alchpot_free_fit(molecule, poly_objs)

In [28]:
molecule_binding[:,4] - alchpot_free_fit

array([0.2773147 , 0.34482303, 0.31873281, 0.38353339, 0.3069185 ,
       0.20597896, 0.22556191, 0.19699205, 0.21242343, 0.21377284,
       0.26464182, 0.26387176])

In [3]:
alchpot_free_fit

array([-6.32045064, -7.07731643, -7.40489704, -7.43887553, -6.60928149,
       -6.75351384, -5.28561609, -5.53941911, -5.51534051, -6.2181486 ,
       -6.11637367, -6.36399776, -6.44287539, -6.36874533, -5.79694313,
       -5.58703834, -5.69732752, -5.9586382 , -5.67481015, -5.92336924])

In [14]:
alchpot_free_explicit - alchpot_free_fit

array([-0.01907362, -0.04020156, -0.05180529, -0.03152655, -0.01131269,
        0.00189264, -0.00966   ,  0.01726984, -0.05133137,  0.02689874,
        0.02828913, -0.04854256])